# **Capstone Project - The Best Neighborhood to Live In**
## **Table of Content**


1.   Introduction: Business Problem
2.   Data
3.   Methodology
4.   Results and Discussion
5.   Conclusion



## **Introduction: Business Problem**

When people are looking for a change of apartment, one of the most important
deciding factors is the neighborhood.
*   Some people might be on a budget while some might be looking for something luxurious
*   A few families would prefer having schools nearby
*   While, some just might be foodies who would love to have many restaurants near their house

We will use our data science abilities to predict the most appropriate neighborhoods for people with the above mentioned preferences.

Here, the **target audience would be people looking for apartments.**

The factors which influence our problem are:


1.   Number of schools in each neighborhood
2.   Number of restaurants in each neighborhood
3.   Budget of prospective tenant






## **Data**
Following data sources have been used to extract/generate the required information:

1.Foursquare API data: to obtain details of restaurants, schools etc. in each neighborhood

 2.Dataset from Kaggle:https://catalog.data.gov/dataset/new-york-city-population-by-neighborhood-tabulation-areas-f2447 
 
 3.https://www.kaggle.com/jackcook/neighborhoods-in-new-york/data : Will be using to obtain population statistics of neighborhood in New York
 
 4.https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas-NTA-/cpf4-rkhq : To obtain geospatial data of neighborhood in New York.

5.https://data.cityofnewyork.us/Housing-Development/Housing-New-York-Units-by-Building/hg8x-zxpr : To obtain information regarding housing units in New York


Installing the required libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed and imported!


Getting Coordinates of New York

In [4]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of New York are',latitude ,longitude)

The coordinates of New York are 40.7127281 -74.0060152


The Map of New York

In [6]:
nyc_map = folium.Map(location=[latitude,longitude], zoom_start=12 )
nyc_map

In [0]:
#Dataset containing details about size of each neighborhood
df = pd.read_csv('nynta.csv')

In [0]:
df.head()

,BoroCode,BoroName,the_geom,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area
0,3,Brooklyn,MULTIPOLYGON (((-73.97604935657381 40.63127590...,47,BK88,Borough Park,39247.227831,5.400502e+07
1,4,Queens,MULTIPOLYGON (((-73.80379022888098 40.77561011...,81,QN51,Murray Hill,33266.904797,5.248828e+07
2,4,Queens,MULTIPOLYGON (((-73.8610972440186 40.763664477...,81,QN27,East Elmhurst,19816.712074,1.972685e+07
3,4,Queens,MULTIPOLYGON (((-73.75725671509139 40.71813860...,81,QN07,Hollis,20976.335574,2.288777e+07
4,1,Manhattan,MULTIPOLYGON (((-73.94607828674226 40.82126321...,61,MN06,Manhattanville,17040.685413,1.064708e+07


In [0]:
df['Shape_Area'].max()
print(df.loc[df.Shape_Area == 327768572.857,['NTAName','Shape_Area']])

                               NTAName    Shape_Area
176  New Springville-Bloomfield-Travis  3.277686e+08


**Largest Neighborhood in New York** : New Springville-Bloomfield-Travis

Population Statistics for Each Neighborhood

In [0]:
df_pop = pd.read_csv('NYC pop.csv')
df_pop.head()

,Borough,Year,FIPS County Code,NTA Code,NTA Name,Population
0,Bronx,2000,5,BX01,Claremont-Bathgate,28149
1,Bronx,2000,5,BX03,Eastchester-Edenwald-Baychester,35422
2,Bronx,2000,5,BX05,Bedford Park-Fordham North,55329
3,Bronx,2000,5,BX06,Belmont,25967
4,Bronx,2000,5,BX07,Bronxdale,34309


In [0]:
#Neighborhood with maximum population
df_pop['Population'].max()

136954

In [0]:
print(df_pop.loc[df_pop.Population==136954, ['NTA Name','Population']])

           NTA Name  Population
95  Upper West Side      136954


**Neighborhood in New York with the maximum population** : Upper West Side

### **Using FourSquare API**

In [0]:
CLIENT_ID = '1RJCHNW45W2WNLEBWOF2KULSEXBFSIADLFSBAMDCOVADRGV4' 
CLIENT_SECRET = '2CK3RNSRP4HYE1QI5PT0LMDGCPYFBC2EAX1ESZ4V3HOC5WSC' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1RJCHNW45W2WNLEBWOF2KULSEXBFSIADLFSBAMDCOVADRGV4
CLIENT_SECRET:2CK3RNSRP4HYE1QI5PT0LMDGCPYFBC2EAX1ESZ4V3HOC5WSC


Schools in New York

In [0]:
search_query = 'School'
radius = 500
print(search_query + ' .... OK!')


School .... OK!


In [0]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1RJCHNW45W2WNLEBWOF2KULSEXBFSIADLFSBAMDCOVADRGV4&client_secret=2CK3RNSRP4HYE1QI5PT0LMDGCPYFBC2EAX1ESZ4V3HOC5WSC&ll=40.7127281,-74.0060152&v=20180604&query=School&radius=500&limit=30'

In [0]:
results = requests.get(url).json()
venues = results['response']['venues']


dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b980f0ef964a520952935e3,Murry Bergtraum High School,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",v-1588591595,False,411 Pearl St,Madison Street,40.711182,-74.001385,"[{'label': 'display', 'lat': 40.71118183194915...",426,10038,US,New York,NY,United States,"[411 Pearl St (Madison Street), New York, NY 1...",NaN
1,4d5ab621e2df60fc9f09d4e5,Actors Studio Drama School,"[{'id': '4bf58dd8d48988d1ac941735', 'name': 'C...",v-1588591595,False,1 Pace Plz,btwn Nassau & Gold St,40.711598,-74.005427,"[{'label': 'display', 'lat': 40.71159778861919...",135,10038,US,New York,NY,United States,"[1 Pace Plz (btwn Nassau & Gold St), New York,...",32694453
2,4e37fc7918a8470916d00b74,Lubin School Of Business,"[{'id': '4d4b7105d754a06372d81259', 'name': 'C...",v-1588591595,False,1 Pace Plz,btwn Nassau & Gold St,40.711024,-74.003732,"[{'label': 'display', 'lat': 40.71102372013692...",270,10038,US,New York,NY,United States,"[1 Pace Plz (btwn Nassau & Gold St), New York,...",NaN
3,4dbc221c6a23e294ba25a512,ABI School Of Barbering,"[{'id': '4bf58dd8d48988d1ad941735', 'name': 'T...",v-1588591595,False,Chambers,NaN,40.715366,-74.008153,"[{'label': 'display', 'lat': 40.71536599999999...",344,10013,US,New York,NY,United States,"[Chambers, New York, NY 10013, United States]",NaN
4,4a78d55cf964a5208be61fe3,NYU School of Professional Studies,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",v-1588591595,False,15 Barclay,btwn Broadway and Church St.,40.712634,-74.008312,"[{'label': 'display', 'lat': 40.71263433816652...",194,10279,US,New York,NY,United States,"[15 Barclay (btwn Broadway and Church St.), Ne...",NaN


In [0]:
dataframe.shape[0]

30

There are a total of 30 schools in New York.

In [0]:
df_pc = dataframe['location.postalCode']
df_pc

0     10038
1     10038
2     10038
3     10013
4     10279
5     10038
6     10013
7     10038
8     10038
9       NaN
10    10038
11    10007
12      NaN
13    10007
14      NaN
15    10007
16    10007
17      NaN
18    10007
19    10007
20    10038
21    10038
22      NaN
23    10038
24    10038
25    10038
26    10013
27    10013
28    10013
29    10013
Name: location.postalCode, dtype: object

In [0]:
df_pc.value_counts()

10038    12
10013     6
10007     6
10279     1
Name: location.postalCode, dtype: int64

This indicates that the area which has the Postal Code 10038 is a suitable option for those buyers/tenants whose kids go to school.
The neighborhoods which have the Postal Code 10038 are :


1.   City Hall
2.   Manhattan
3.   Downtown





In [15]:
data1 = [['City Hall',40.7128,-74.0061],['Manhattan',40.7831,-73.9712],['Downtown',40.7209,-74.0007]] 
df2 = pd.DataFrame(data1, columns=['Neighborhood','Lat','Long'])
df2.head()

,Neighborhood,Lat,Long
0,City Hall,40.7128,-74.0061
1,Manhattan,40.7831,-73.9712
2,Downtown,40.7209,-74.0007


Map Showing Top 3 Neighborhoods with **Highest Number of Schools**

In [26]:
r = folium.Map(location=[latitude,longitude], zoom_start=12)
tip = 'Click me!'

folium.Marker([40.7128, -74.0061], popup='<b>City Hall</b>', tooltip=tip).add_to(r)
folium.Marker([40.7831, -73.9712], popup='<b>Manhattan</b>', tooltip=tip).add_to(r)
folium.Marker([40.7209, -74.0007], popup='<b>Downtown</b>', tooltip=tip).add_to(r)


r

Next, we look for neighborhood with a lot of restaurants nearby.

In [0]:
search_query2 = 'restaurant'
radius = 500
print(search_query2 + ' .... OK!')

restaurant .... OK!


In [0]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
venues = results['response']['venues']

df_r = json_normalize(venues)
df_r.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4bc238adf8219c744286b410,Amore's Pizza Restaurant,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1588592593,False,147 Chambers St,Hudson Street,40.715860,-74.009888,"[{'label': 'display', 'lat': 40.71585960614924...",477,10007,US,New York,NY,United States,"[147 Chambers St (Hudson Street), New York, NY...",1431324,https://www.seamless.com/menu/cafe-amores-pizz...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
1,4b4fdfc8f964a520801827e3,TJ Byrnes Bar and Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1588592593,False,77 Fulton St,Gold St,40.709233,-74.003747,"[{'label': 'display', 'lat': 40.70923312629616...",433,10038,US,New York,NY,United States,"[77 Fulton St (Gold St), New York, NY 10038, U...",NaN,NaN,NaN,NaN,NaN,NaN,131643631
2,4c4890da1879c9b6cce6e143,New Shezan Restaurant,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",v-1588592593,False,183 Church St,btwn Duane & Reade St.,40.715789,-74.007227,"[{'label': 'display', 'lat': 40.71578949233164...",355,10007,US,New York,NY,United States,"[183 Church St (btwn Duane & Reade St.), New Y...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b88199af964a520e6df31e3,Win Won Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1588592593,False,10 Liberty Pl,Maiden Lane,40.709193,-74.009344,"[{'label': 'display', 'lat': 40.709193, 'lng':...",483,10038,US,New York,NY,United States,"[10 Liberty Pl (Maiden Lane), New York, NY 100...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45e5c256f964a52046431fe3,Mudville Restaurant & Tap House,"[{'id': '4bf58dd8d48988d14c941735', 'name': 'W...",v-1588592593,False,126 Chambers St,btwn W Broadway & Church St,40.715336,-74.008881,"[{'label': 'display', 'lat': 40.71533575723845...",377,10007,US,New York,NY,United States,[126 Chambers St (btwn W Broadway & Church St)...,299726,https://www.seamless.com/menu/mudville-restaur...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,43156651


In [0]:
df_r.shape[0]

30

There are 30 restaurants in New York.

In [0]:
df_r['location.postalCode'].value_counts()

10007    15
10038     8
10013     5
10004     2
Name: location.postalCode, dtype: int64

In [11]:
data = [['Tribeca', 40.7163,-74.0086], ['Downtown', 40.7209,-74.0007], ['Lower Manhattan', 40.7549,-73.9840]]
df_rest = pd.DataFrame(data, columns = ['Neighborhood', 'Lat','Long'])
df_rest.head() 
  

,Neighborhood,Lat,Long
0,Tribeca,40.7163,-74.0086
1,Downtown,40.7209,-74.0007
2,Lower Manhattan,40.7549,-73.9840


This implies people who prefer eating out alot should look for apartments in the neighborhoods with postal code 10007. The neighborhoods which have 10007 postal code are : 


1.   Tribeca
2.   Downtown
3.   Lower Manhattan



Map displaying Top 3 Neighborhoods with highest **Number of Restaurants**

In [18]:
m = folium.Map(
    location=[latitude,longitude],
    zoom_start=12,
    
)

tooltip = 'Click me!'

folium.Marker([40.7549, -73.9840], popup='<b>Lower Manhattan</b>', tooltip=tooltip).add_to(m)
folium.Marker([40.7163, -74.0086], popup='<b>Tribeca</b>', tooltip=tooltip).add_to(m)
folium.Marker([40.7209, -74.0007], popup='<b>Downtown</b>', tooltip=tooltip).add_to(m)


m

Classifying the neighborhood on the basis of how many **Extremely Low Income Units and Middle Income Units** they have.

In [0]:
df_b = pd.read_csv('Housing_New_York_Units_by_Building.csv')
df_b.head()

,Project ID,Project Name,Program Group,Project Start Date,Project Completion Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,Community Board,Council District,Census Tract,NTA - Neighborhood Tabulation Area,Latitude,Longitude,Latitude (Internal),Longitude (Internal),Building Completion Date,Reporting Construction Type,Extended Affordability Only,Prevailing Wage Status,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Studio Units,1-BR Units,2-BR Units,3-BR Units,4-BR Units,5-BR Units,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units
0,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,03/31/2020,NaN,1564.0,83,2 AVENUE,Manhattan,10003.0,1.004600e+09,1006591.0,MN-03,2.0,38,MN22,40.726642,-73.989101,40.72676,-73.989368,03/31/2020,Preservation,No,Non Prevailing Wage,0,4,0,0,0,0,0,0,0,4,0,0,0,0,1,3,4,4
1,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,03/31/2020,NaN,10537.0,13,EAST 3 STREET,Manhattan,10003.0,1.004590e+09,1077971.0,MN-03,2.0,38,MN22,40.726061,-73.990984,40.72620,-73.990811,03/31/2020,Preservation,No,Non Prevailing Wage,0,10,0,0,0,0,2,6,2,0,0,0,0,0,1,9,10,10
2,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,03/31/2020,NaN,10557.0,21,EAST 3 STREET,Manhattan,10003.0,1.004590e+09,1077972.0,MN-03,2.0,38,MN22,40.725932,-73.990677,40.72608,-73.990486,03/31/2020,Preservation,No,Non Prevailing Wage,15,1,0,0,0,0,6,5,5,0,0,0,0,0,5,11,16,16
3,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,03/31/2020,NaN,10569.0,23,EAST 3 STREET,Manhattan,10003.0,1.004590e+09,1077973.0,MN-03,2.0,38,MN22,40.725901,-73.990602,40.72605,-73.990403,03/31/2020,Preservation,No,Non Prevailing Wage,14,3,0,0,0,0,8,6,3,0,0,0,0,0,3,14,17,17
4,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,03/31/2020,NaN,10580.0,25,EAST 3 STREET,Manhattan,10003.0,1.004590e+09,1077974.0,MN-03,2.0,38,MN22,40.725868,-73.990526,40.72601,-73.990327,03/31/2020,Preservation,No,Non Prevailing Wage,17,1,0,0,0,0,9,7,2,0,0,0,0,0,2,16,18,18


In [0]:
df_l = df_b['Extremely Low Income Units']
df_l

0        0
1        0
2       15
3       14
4       17
        ..
4546     0
4547     0
4548     0
4549     0
4550     0
Name: Extremely Low Income Units, Length: 4551, dtype: int64

In [0]:
df_l.sort_values(ascending=False,axis=0)

3891    429
1031    250
1490    247
1379    241
486     239
       ... 
3004      0
3003      0
3002      0
3000      0
0         0
Name: Extremely Low Income Units, Length: 4551, dtype: int64

Top 5 neighborhoods with maximum number of extremely low income housing units

In [0]:
df_b.iloc[[3891,1031,1490,1379,486]]

,Project ID,Project Name,Program Group,Project Start Date,Project Completion Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,Community Board,Council District,Census Tract,NTA - Neighborhood Tabulation Area,Latitude,Longitude,Latitude (Internal),Longitude (Internal),Building Completion Date,Reporting Construction Type,Extended Affordability Only,Prevailing Wage Status,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Studio Units,1-BR Units,2-BR Units,3-BR Units,4-BR Units,5-BR Units,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units
3891,55396,North Shore Plaza_ML,Multifamily Finance Program,12/18/2014,12/17/2016,746078.0,85,HOLLAND AVENUE,Staten Island,10303.0,5.012720e+09,5109139.0,SI-01,49.0,31902,SI12,40.638543,-74.171232,40.63910,-74.170235,12/17/2016,Preservation,No,Non Prevailing Wage,429,0,107,0,0,0,0,0,0,0,0,0,0,536,536,0,536,536
1031,65341,FRIENDSET APTS HDFC.HUDMF.FY18,Multifamily Finance Program,12/20/2018,NaN,336169.0,3528,MERMAID AVENUE,Brooklyn,11224.0,3.070460e+09,3189510.0,BK-13,47.0,342,BK21,40.574821,-74.001044,40.57430,-74.001127,NaN,Preservation,Yes,Non Prevailing Wage,250,4,1,4,0,0,0,241,18,0,0,0,0,0,259,0,259,259
1490,62209,ADEE TOWER APARTMENTS INC.HRP.FY18,Multifamily Finance Program,06/11/2018,NaN,52029.0,3000,BRONX PARK EAST,Bronx,10467.0,2.043360e+09,NaN,BX-27,15.0,334,BX99,40.856665,-73.875798,40.86224,-73.873047,NaN,Preservation,No,Non Prevailing Wage,247,45,0,0,0,0,0,3,96,193,0,0,0,0,0,292,292,292
1379,67426,RUPPERT HOUSE,Multifamily Finance Program,06/27/2018,06/27/2018,804760.0,222,EAST 93 STREET,Manhattan,10128.0,1.015380e+09,1089873.0,MN-08,5.0,154,MN32,40.782965,-73.950061,40.78231,-73.949469,06/27/2018,Preservation,Yes,Non Prevailing Wage,241,163,0,0,0,1,84,163,158,0,0,0,0,0,0,405,405,405
486,68177,CAPITOL APARTMENTS HDFC.HUDMF,Multifamily Finance Program,06/28/2019,06/28/2019,4540.0,834,8 AVENUE,Manhattan,10019.0,1.010220e+09,1024813.0,MN-05,3.0,131,MN17,40.762505,-73.985936,40.76253,-73.985517,06/28/2019,Preservation,Yes,Non Prevailing Wage,239,6,2,3,27,1,88,190,0,0,0,0,0,0,278,0,278,278


Hence, people who are **tight on a budget** should look for apartments in these 5 neighborhoods.

Map displaying Top 5 neighborhoods with highest number of **Extremely Low Income Units**

In [27]:
e = folium.Map(
    location=[latitude,longitude],
    zoom_start=10,
    
)

tooltip = 'Click me!'

folium.Marker([40.63910,-74.170235], popup='<b>Annadale-Huguenot-Princes Bay-Eltingville</b>', tooltip=tooltip).add_to(e)
folium.Marker([40.574821,-74.001044], popup='<b>Seagate-Coney Island</b>', tooltip=tooltip).add_to(e)
folium.Marker([40.856665,-73.875798], popup='<b>park-cemetery-etc-Bronx</b>', tooltip=tooltip).add_to(e)
folium.Marker([40.782965,-73.950061], popup='<b>Yorkville</b>', tooltip=tooltip).add_to(e)
folium.Marker([40.762505,-73.985936], popup='<b>Midtown-Midtown South</b>', tooltip=tooltip).add_to(e)
e

In [0]:
df_m = df_b['Middle Income Units']
df_m.sort_values(ascending=False, axis=0)

2466    448
4267    422
2482    352
4082    343
1010    284
       ... 
2896      0
2894      0
2893      0
2887      0
0         0
Name: Middle Income Units, Length: 4551, dtype: int64

Top 5 neighborhood with **maximum number of Middle Income Housing Units**

In [0]:
df_b.iloc[[2466,4267,2482,4082,1010]]

,Project ID,Project Name,Program Group,Project Start Date,Project Completion Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,Community Board,Council District,Census Tract,NTA - Neighborhood Tabulation Area,Latitude,Longitude,Latitude (Internal),Longitude (Internal),Building Completion Date,Reporting Construction Type,Extended Affordability Only,Prevailing Wage Status,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Studio Units,1-BR Units,2-BR Units,3-BR Units,4-BR Units,5-BR Units,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units
2466,52774,JAMAICA CROSSING (HIGH-RISE),Multifamily Finance Program,12/22/2016,NaN,974242.0,147-40,ARCHER AVENUE,Queens,11435.0,4.099980e+09,4214004.0,QN-12,27.0,208,QN61,40.700836,-73.806551,40.70053,-73.806783,NaN,New Construction,No,Non Prevailing Wage,0,0,90,0,448,1,98,202,217,22,0,0,0,0,539,0,539,539
4267,55506,Cadman Tower,Multifamily Finance Program,06/26/2014,06/26/2014,221260.0,79,CLARK STREET,Brooklyn,11201.0,3.002320e+09,3001727.0,BK-02,33.0,1,BK09,40.697436,-73.992881,40.69761,-73.992171,06/26/2014,Preservation,No,Non Prevailing Wage,0,0,0,0,422,0,0,0,0,0,0,0,0,422,0,422,422,422
2482,63371,Scott Tower (HDC),Multifamily Finance Program,12/20/2016,NaN,101663.0,3400,PAUL AVENUE,Bronx,10468.0,2.032510e+09,2015444.0,BX-07,11.0,409,BX28,40.877190,-73.889569,40.87744,-73.888885,NaN,Preservation,No,Non Prevailing Wage,0,0,0,0,352,0,2,60,206,84,0,0,0,0,0,352,352,352
4082,55511,Arverne/Nordak,Multifamily Finance Program,09/23/2014,NaN,811201.0,353,BEACH 57 STREET,Queens,11692.0,4.158950e+09,4436427.0,QN-14,31.0,97204,QN12,40.593738,-73.787140,40.59363,-73.787990,NaN,Preservation,No,Non Prevailing Wage,0,0,0,0,343,0,0,0,0,0,0,0,0,343,0,343,343,343
1010,61931,HUNTERS POINT SOUTH PARCEL C -NORTH TOWER,Multifamily Finance Program,12/26/2018,NaN,981672.0,52-03,CENTER BOULEVARD,Queens,11101.0,4.000060e+09,NaN,QN-02,26.0,1,QN31,40.742166,-73.960033,40.74191,-73.959553,NaN,New Construction,No,Non Prevailing Wage,0,160,0,90,284,1,174,145,216,0,0,0,0,0,535,0,535,800


Therefore, people who are **willing to pay above than average rent**, should look for apartments in these 5 neighbourhoods.

Map displaying Top 5 Neighborhoods with Highest Number of **Middle Income Housing Units**

In [28]:
i = folium.Map(
    location=[latitude,longitude],
    zoom_start=10,
    
)

tooltip = 'Click me!'

folium.Marker([40.700836,-73.806551], popup='<b>Hammels-Arverne-Edgemere</b>', tooltip=tooltip).add_to(i)
folium.Marker([40.697436,-73.992881], popup='<b>Brooklyn</b>', tooltip=tooltip).add_to(i)
folium.Marker([40.877190,-73.889569], popup='<b>Bronxdale</b>', tooltip=tooltip).add_to(i)
folium.Marker([40.593738,-73.787140], popup='<b>Queens</b>', tooltip=tooltip).add_to(i)
folium.Marker([40.742166,-73.960033], popup='<b>Springfield Gardens North</b>', tooltip=tooltip).add_to(i)
i

## **Methodology**

In this project, our main aim is to shortlist neighborhoods as per the requirements of the prospective tenants. Some of the classifying factors include the rent, the number of schools, the number of restaurants etc.

Using Foursquare API data, we have first found the **total number of schools and restaurants** in each neighborhood in New York. 

Then, we have shortlisted the areas having the maximum number of schools and restaurants **based on their postal codes in order to maintain accuracy**.

Next, we have performed analysis on the NYC Housing Data and classified neighborhoods on the basis of how many extremely low income units and how many middle income units they have.

Finally, we have **represented all these shortlisted locations on the New York map according to each criteria respectively**.




## **Results and Discussion**

Upon analysing all the datasets we observe that there many neighborhoods in New York, each with its pros and cons. 

Neighborhood with the **largest population**: Upper West Side

Neighborhood with the **largest area**: New Springville-Bloomfield-Travis

For **people on a budget**, the following neighborhoods are best suited:



1.   Annadale-Huguenot-Princes Bay-Eltingville
2.   Seagate-Coney Island
3.   Park-cemetery-etc-Bronx
4.   Yorkville
5.   Midtown-Midtown South


Prospective tenants **willing to pay above average rents** should look for an apartment in the following neighborhoods:



1.   Hammels-Arverne-Edgemere
2.   Brooklyn
3.   Bronxdale
4.   Queens
5.   Springfield Gardens North




We see that for **prospective tenants with kids**, the following 3 neighborhoods are the best suited:


1.   City Hall
2.   Manhattan
3.   Downtown

For people looking for an apartment **with alot of restaurants nearby**, the following neighborhoods are best suited:



1.   Tribeca
2.   Lower Manhattan
3.   Downtown







## **Conclusion**

The purpose of this project is to help prospective tenants find the right neighborhood for themselves. 

We have shortlisted a few neighborhoods according to the 3 criterias; schools, restaurants and budget. There are many more classifying criteria which can be used if the datasets for the same are available.

This is **not an exhaustive analysis** as the definition of the Best Neighborhood to Live In is different for every individual and the data for every criteria is not accessible. 

The final decision on an optimal neighborhood can only be made by the prospective tenant after taking into consideration other factors such as proximity to public transport facilities, healthcare facilities, cleanliness, climate etc.

This analysis will surely help the prospective tenants by giving them a rough outline and **a place to start their search of an apartment from**.

